In [1]:
import logging
import pickle
import json
import time

from pathlib import Path
import uuid
from timeit import default_timer as timer

from geoloc_imc_2023.cbg import CBG, get_prefix_from_ip
from geoloc_imc_2023.query_api import get_measurements_from_tag

from geoloc_imc_2023.default import (
    PROBES_PATH,
    ANCHORS_PATH,
    HITLIST_PATH,
)

logging.basicConfig(level=logging.INFO)

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

NB_TARGET = 3
NB_VP = 10

In [2]:
# load hitlist
with open(HITLIST_PATH, "rb") as f:
    targets_per_prefix = pickle.load(f)

# load probes
with open(PROBES_PATH, "rb") as f:
    probes = pickle.load(f)

# load anchors
with open(ANCHORS_PATH, "rb") as f:
    anchors = pickle.load(f)

## select target and vps dataset

In [3]:
# select targets and vps from anchors
targets= list(anchors.keys())[:NB_TARGET]
vps = {}
for i, probe in enumerate(probes):
    if i >= NB_VP: break
    vps[probe] = probes[probe]

logging.info(f"nb targets: {len(targets)} : {[target for target in targets]}")
logging.info(f"nb_vps : {len(vps)} {[vp for vp in vps]}")

INFO:root:nb targets: 10819 : ['45.138.229.91', '77.174.76.85', '82.217.219.124', '83.81.82.33', '193.0.0.78', '87.3.187.45', '77.174.30.45', '93.108.63.51', '86.89.224.211', '216.147.123.181', '76.82.152.84', '2.57.252.147', '81.220.141.64', '85.225.8.64', '88.163.164.208', '77.95.64.208', '185.26.198.33', '82.65.164.206', '81.187.145.168', '83.79.38.169', '49.0.64.43', '24.22.249.162', '213.136.0.6', '212.159.71.10', '193.63.209.30', '73.192.147.189', '192.108.131.3', '87.104.43.138', '84.214.144.170', '195.113.147.190', '70.95.176.202', '193.29.220.38', '89.101.70.74', '87.238.56.246', '195.170.160.30', '217.31.48.2', '212.25.179.151', '93.190.128.179', '5.133.42.106', '83.85.81.24', '194.160.23.3', '77.236.64.30', '45.80.169.238', '109.210.115.213', '95.90.194.169', '185.78.48.181', '76.154.155.85', '212.159.52.129', '59.167.217.120', '147.230.16.42', '193.0.3.38', '195.34.46.43', '152.171.211.107', '193.206.159.234', '176.93.226.47', '91.203.72.35', '82.0.207.96', '83.191.164.214'

## Tier 1: probing each target prefixes

In [ ]:
measurement_uuid = uuid.uuid4()
dry_run = True
start_time = time.time()

# measurement configuration for retreival
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "mesurement towards all anchors with all probes as vps",
    "type" : "target prefix",
    "af" : 4,
    "start_time": start_time,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
measurement_config_path = Path(".") / f"../measurements_metadata/{str(measurement_uuid)}.json"
with open(measurement_config_path, "w") as f:
    json.dump(measurement_config, f)

cbg = CBG(ripe_credentials)

# get target prefixes
target_prefixes = []
for target_addr in targets:
    target_prefix = get_prefix_from_ip(target_addr)
    target_prefixes.append(target_prefix)

# measurement for 3 targets in every target prefixes
prefix_measurement_ids = cbg.prefix_probing(
    target_prefixes= target_prefixes, 
    vps= vps, 
    targets_per_prefix= targets_per_prefix,
    tag= measurement_uuid,
    dry_run=True
)
 
logging.info(f"measurement : {measurement_uuid} done")

measurement_config["end_time"] = time.time()

# retreive measurement results from RIPE Atlas
for i, (prefix, ids) in enumerate(prefix_measurement_ids.items()):
    logging.info(f"{prefix} : {ids}")

## Retrieve prefix probing results 

In [1]:
measurement_results = get_measurements_from_tag(measurement_config["UUID"], ripe_credentials["key"])

In [ ]:
# test that everything is alright
logging.info(f"nb measurements retrieved: {len(measurement_results)}")

for i, measurement_result in enumerate(measurement_results):
    

In [2]:
# save results
out_file = f"../results/{measurement_config['UUID']}.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

../results/init_measurement.pickle


## Tier 2: probe each target

In [1]:
ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

# load hitlist
hitlist_file = Path(".") / "../datasets/parsed_hitlist.pickle"
with open(hitlist_file, "rb") as f:
    hitlist_addr = pickle.load(f)

targets = [anchor_addr for anchor_addr in anchors][:NB_TARGET]
vps = [anchor_addr for anchor_addr in anchors][:NB_VP]

logging.info(f"nb targets: {len(targets)}")
logging.info(f"nb_vps : {len(vps)}")

In [ ]:
cbg = CBG(targets, vps, ripe_credentials, hitlist_addr, anchors)

start = timer()
measurement_ids = await cbg.target_probing(targets, vps)
end =timer()

time_elapsed = end - start
print(time_elapsed)

In [ ]:
# save results
out_file = f"../results/target_measurement_ids.pickle"
with open(out_file, "wb") as f:
    pickle.dump(measurement_ids,f)

## Retreive target measurements

In [ ]:
# load hitlist
in_file = Path(".") / "../results/target_measurement_ids.pickle"
with open(in_file, "rb") as f:
    measurement_ids = pickle.load(f)

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

print(len(measurement_ids))

measurement_results = []
for measurement_id in measurement_ids:
    measurement_result = retreive_single_measurement(measurement_id, anchors, ripe_credentials["key"])
    print(f"got results for target: {measurement_result['target_addr']} from {len(measurement_result['results'])} servers")
    measurement_results.append(measurement_result)

In [15]:
# save results
out_file = f"../results/target_measurement.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

../results/target_measurement.pickle
